## Join Regionalização LOA

Neste notebook fazemos o join dos datasets gerados nos notebooks 1 e 2.

Esse join é realizado apenas para as dotações da LOA que representam emendas de vereador.

In [1]:
import pandas as pd
import os

from config import DATA_FOLDER

## carregar dados

In [2]:
df_loa = pd.read_csv(os.path.join(DATA_FOLDER, 'df_loa.csv'), low_memory=False, sep=';')
df_da = pd.read_csv(os.path.join(DATA_FOLDER, 'df_da.csv'), low_memory=False, sep=';')

In [3]:
df_loa.head()

,exercicio,cod_emenda,direta_ou_indireta,cod_orgao,cod_unidade,cod_programa,cod_funcao,cod_subfuncao,cod_proj_atividade,cod_fonte,desc_orgao,desc_unidade,desc_programa,desc_funcao,desc_subfuncao,desc_proj_atividade,desc_fonte,valor_orcado,index,is_emenda
0,2025,NaN,Administraçăo Direta,84,10,3003,10,304,2522,2,Fundo Municipal de Saúde,Fundo Municipal de Saúde,"Açőes e Serviços da Saúde em Atençăo Básica, E...",Saúde,Vigilância Sanitária,Manutençăo e Operaçăo de Vigilância em Saúde,Transferęncias Federais,76700000,2025.84.10.2522,False
1,2025,NaN,Administraçăo Direta,94,10,3005,18,541,7117,8,Fundo Especial do Meio Ambiente e Desenvolvime...,Fundo Especial do Meio Ambiente e Desenvolvime...,Promoçăo da Sustentabilidade Ambiental,Gestăo Ambiental,Preservaçăo e Conservaçăo Ambiental,Recuperaçăo de áreas degradadas e/ou contaminadas,Tesouro Municipal - Recursos Vinculados,310000,2025.94.10.7117,False
2,2025,NaN,Administraçăo Direta,94,10,3005,18,541,7117,8,Fundo Especial do Meio Ambiente e Desenvolvime...,Fundo Especial do Meio Ambiente e Desenvolvime...,Promoçăo da Sustentabilidade Ambiental,Gestăo Ambiental,Preservaçăo e Conservaçăo Ambiental,Recuperaçăo de áreas degradadas e/ou contaminadas,Tesouro Municipal - Recursos Vinculados,400000,2025.94.10.7117,False
3,2025,NaN,Administraçăo Direta,20,50,3009,26,453,1800,1,Secretaria Municipal de Mobilidade e Trânsito,Secretaria Executiva de Transporte e Mobilidad...,Melhoria da Mobilidade Urbana Universal,Transporte,Transportes Coletivos Urbanos,Eletrificaçăo da frota de veículos do Sistema ...,Operaçőes de Crédito,7071768,2025.20.50.1800,False
4,2025,NaN,Administraçăo Direta,20,50,3009,26,453,1800,1,Secretaria Municipal de Mobilidade e Trânsito,Secretaria Executiva de Transporte e Mobilidad...,Melhoria da Mobilidade Urbana Universal,Transporte,Transportes Coletivos Urbanos,Eletrificaçăo da frota de veículos do Sistema ...,Operaçőes de Crédito,19229040,2025.20.50.1800,False


## Selecionando apenas emendas

Ao todo são mil emendas nos 4 anos da legislatura. O  dataset que estamos construindo trabalha apenas com elas.

In [7]:
df_loa['is_emenda'].mean()

np.float64(0.0551508283416977)

In [9]:
df_loa['is_emenda'].sum()

np.int64(1022)

In [10]:
df_loa = df_loa[df_loa['is_emenda']].copy().reset_index(drop=True)

In [14]:
df_loa['is_emenda'].all()

np.True_

In [15]:
df_loa.drop('is_emenda', axis=1, inplace=True)

In [16]:
df_loa['cod_emenda'].isnull().any()

np.False_

In [17]:
df_loa['cod_emenda'].sample(5)

1002      95.0
365     3520.0
903     1644.0
743     6627.0
254     3514.0
Name: cod_emenda, dtype: float64

In [18]:
#posso transformar em int agora porque não tem mais NAN
df_loa['cod_emenda'] = df_loa['cod_emenda'].astype(int)

In [13]:
df_loa.groupby('exercicio').count()['cod_emenda']

exercicio
2022    211
2023    250
2024    302
2025    259
Name: cod_emenda, dtype: int64

## Join dataframes

Agora vamos fazer join nos dataframes com base nos indices que foram criados.

Os dados das emendas (df_loa) ficam na esquerda, e é um left join. Ou seja: estamos buscando as regionalizações das emendas.

In [4]:
df_loa.dtypes

exercicio                int64
cod_emenda             float64
direta_ou_indireta      object
cod_orgao                int64
cod_unidade              int64
cod_programa             int64
cod_funcao               int64
cod_subfuncao            int64
cod_proj_atividade       int64
cod_fonte                int64
desc_orgao              object
desc_unidade            object
desc_programa           object
desc_funcao             object
desc_subfuncao          object
desc_proj_atividade     object
desc_fonte              object
valor_orcado             int64
index                   object
is_emenda                 bool
dtype: object

In [5]:
df_da.dtypes

cod_regiao               int64
cod_subprefeitura        int64
cod_distrito             int64
desc_regiao             object
desc_subprefeitura      object
desc_distrito           object
valor_regionalizado    float64
index                   object
dtype: object

In [19]:
df_merge = pd.merge(df_loa, df_da, how='left', on='index', suffixes=('', '_da'))
df_merge.head()

,exercicio,cod_emenda,direta_ou_indireta,cod_orgao,cod_unidade,cod_programa,cod_funcao,cod_subfuncao,cod_proj_atividade,cod_fonte,...,desc_fonte,valor_orcado,index,cod_regiao,cod_subprefeitura,cod_distrito,desc_regiao,desc_subprefeitura,desc_distrito,valor_regionalizado
0,2025,3514,Administraçăo Direta,60,10,3001,13,392,9222,0,...,Tesouro Municipal,1000,2025.60.10.9222,0,0,0,Supra-Regional,Supra Subprefeitura,Supra-Distrital,1000.0
1,2025,1791,Administraçăo Direta,60,10,3022,15,452,9241,0,...,Tesouro Municipal,2000000,2025.60.10.9241,0,0,0,Supra-Regional,Supra Subprefeitura,Supra-Distrital,2000000.0
2,2025,3514,Administraçăo Direta,60,10,3017,27,812,9223,0,...,Tesouro Municipal,1000,2025.60.10.9223,0,0,0,Supra-Regional,Supra Subprefeitura,Supra-Distrital,1000.0
3,2025,3549,Administraçăo Direta,20,10,3009,26,453,9145,0,...,Tesouro Municipal,100000,2025.20.10.9145,0,0,0,Supra-Regional,Supra Subprefeitura,Supra-Distrital,100000.0
4,2025,3533,Administraçăo Direta,20,10,3009,26,572,9138,0,...,Tesouro Municipal,30000,2025.20.10.9138,0,0,0,Supra-Regional,Supra Subprefeitura,Supra-Distrital,30000.0


In [20]:
df_merge['desc_regiao'].isnull().any()

np.False_

In [21]:
df_merge['desc_regiao'].value_counts()

desc_regiao
Supra-Regional    906
Sul                76
Leste              71
Norte              14
Oeste               8
Centro              2
Name: count, dtype: int64

In [22]:
df_merge.to_csv(os.path.join(DATA_FOLDER, 'df_merge_loa_da.csv'), sep=';', index=False)